In [1]:
from datetime import datetime
import xarray as xr

from mhwpype import FixedBaseline
from mhwpype.utils import update_names, reformat_longitude, assign_depth
from mhwpype.core import build_heat_spike_mask, build_mhw_mask, group_from_mask, group_stats

In [2]:
filepath = 'data/sst.day.mean.1981-2024.nc'
ds = xr.open_dataset(filepath)

ds = update_names(ds)
ds['longitude'] = reformat_longitude(ds.longitude)
ds = ds.sortby(['time','latitude','longitude'])
ds = ds.sel(latitude= slice(41.5,46.5), longitude=slice(-130,-122))
ds = assign_depth(ds,0)

# Dataset of interest
dsoi = ds.sel(time = slice(datetime(2015,1,1), datetime(2024,12,31,23,59,59))) 

In [3]:
reference_begin = datetime(1982,1,1,0,0,0)
reference_end = datetime(2011,12,31,23,59,59)
Fixed = FixedBaseline(reference_begin_datetime=reference_begin, reference_end_datetime=reference_end)

In [4]:
climatology = Fixed.build_daily_climatology(ds.sea_water_temperature, reset_to_input_time=False)

In [5]:
threshold = Fixed.build_daily_threshold(ds.sea_water_temperature, reset_to_input_time=False)

In [6]:
hs_mask = build_heat_spike_mask(dsoi.sea_water_temperature, threshold)

In [7]:
mhw_mask = build_mhw_mask(dsoi.sea_water_temperature, threshold)

In [8]:
mhw_groups = group_from_mask(mhw_mask)

In [9]:
mhw_stats = group_stats(dsoi.sea_water_temperature, climatology, threshold, mhw_groups)

In [10]:
mhw_stats = mhw_stats.sel(event_id = slice(datetime(2015,1,1), datetime(2024,12,31,23,59,59)))

In [11]:
mhw_stats

<xarray.Dataset> Size: 215MB
Dimensions:                                (event_id: 1788, depth: 1,
                                            latitude: 20, longitude: 25,
                                            quantile: 3)
Coordinates:
  * depth                                  (depth) int64 8B 0
  * event_id                               (event_id) datetime64[ns] 14kB 201...
  * latitude                               (latitude) float32 80B 41.62 ... 4...
  * longitude                              (longitude) float32 100B -129.9 .....
  * quantile                               (quantile) float64 24B 0.9 0.95 0.99
Data variables:
    event_start                            (event_id, depth, latitude, longitude, quantile) datetime64[ns] 21MB ...
    event_end                              (event_id, depth, latitude, longitude, quantile) datetime64[ns] 21MB ...
    event_duration                         (event_id, depth, latitude, longitude, quantile) float64 21MB ...
    event_peak_sea_water_temperature       (event_id, depth, latitude, longitude, quantile) float64 21MB ...
    event_peak_sea_water_temperature_time  (event_id, depth, latitude, longitude, quantile) datetime64[ns] 21MB ...
    event_peak_anomaly                     (event_id, depth, latitude, longitude, quantile) float64 21MB ...
    event_peak_anomaly_time                (event_id, depth, latitude, longitude, quantile) datetime64[ns] 21MB ...
    event_intensity_max                    (quantile, event_id, depth, latitude, longitude) float32 11MB ...
    event_intensity_mean                   (quantile, event_id, depth, latitude, longitude) float32 11MB ...
    event_intensity_stdev                  (quantile, event_id, depth, latitude, longitude) float32 11MB ...
    event_intensity_cumulative             (quantile, event_id, depth, latitude, longitude) float32 11MB ...
    event_peak_category                    (event_id, depth, latitude, longitude, quantile) float64 21MB ...